In [1]:
from enum import Enum

def CopyClientData(lakeName, instance: Enum):
  print("LakeName:", lakeName)
  print("Instance:", instance.name)
  print("InstanceId:", instance.value)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('companies', 'client.'+instance.name)
  print("Reading data since", LastSyncDate)  
  
  print("Load companies data")
  PrepareDataBronze (
    lakeName = lakeName,
    instance = instance, 
    entity = BronzeTable.Companies_Client,
    entityName = 'tmpCompanies',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
          CompanyId
        , """+str(instance.value)+""" as InstanceId
        , CompanyName
        , ContactName
        , Phone
        , ContactEmail
        , AddressLine2
        , AddressLine1
        , City
        , RegionName
        , PostalCode
        , CountryCode
        , SubRegionName
        , RegionCode
        , SubRegionCode
        , UserIntegrationType
        , Market
        , EnableCustomerDW
        , ModifiedUtc
        , current_timestamp() as SilverModifiedUtc
    from tmpCompanies 
""")  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into silver.Companies as t
    using SourceData as s
      on t.CompanyId = s.CompanyId and t.InstanceId = s.InstanceId
    when matched then update set 
          CompanyName = s.CompanyName
        , ContactName = s.ContactName
        , Phone = s.Phone
        , ContactEmail = s.ContactEmail
        , AddressLine2 = s.AddressLine2
        , AddressLine1 = s.AddressLine1
        , City = s.City
        , RegionName = s.RegionName
        , PostalCode = s.PostalCode
        , CountryCode = s.CountryCode
        , SubRegionName = s.SubRegionName
        , RegionCode = s.RegionCode
        , SubRegionCode = s.SubRegionCode
        , UserIntegrationType = s.UserIntegrationType
        , Market = s.Market
        , EnableCustomerDW = s.EnableCustomerDW
        , Modified = s.ModifiedUtc
        , SilverModifiedUtc = s.SilverModifiedUtc
    when not matched then 
      insert ( CompanyId, InstanceId, InstanceName, AccountId, AccountName, TierId, TierName, Status, IsDeleted, IsActive, BillingCreatedUtc, IsPending, NonCab,  
               SetupCompletionTime, VerticalId, VerticalName, OperationalCreatedUtc, DataPurgeCompleted,  
               CompanyName, ContactName, Phone, ContactEmail, AddressLine2, AddressLine1, City, RegionName, PostalCode, CountryCode, SubRegionName, 
               RegionCode, SubRegionCode, UserIntegrationType, Market, EnableCustomerDW, Modified, SilverModifiedUtc )
      values ( CompanyId, InstanceId, null, null, null, null, null, null, null, null, null, null, null,
               null, null, null,  null, null, 
               CompanyName, ContactName, Phone, ContactEmail, AddressLine2, AddressLine1, City, RegionName, PostalCode, CountryCode, SubRegionName, 
               RegionCode, SubRegionCode, UserIntegrationType, Market, EnableCustomerDW, ModifiedUtc, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"ModifiedUtc": "max"}).collect()[0][0]
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'companies', 'client.'+instance.name)

In [2]:
# PrepareData (
#     fileSystem = 'bronze',
#     dataSource = 'cab/operational',
#     instanceName = 'att', 
#     tableName = 'companies_client', 
#     pkColumns = 'CompanyId', 
#     modificationTimeColumn = 'ModifiedUtc',
#     startDateStr = '2000-03-01T00:00:00.0',
#     entityName = 'Companies'
#   )

In [3]:
%sql
-- select * from Companies

RowNum,CompanyId,CompanyName,CreatedUtc,ModifiedUtc,ContactName,ContactEmail,AddressLine2,AddressLine1,City,RegionName,PostalCode,Phone,Tier,CountryCode,SubRegionName,RegionCode,SubRegionCode,UserIntegrationType,Market,MarketBaseUrl,Deleted,EnableCustomerDW,year,month,day
1,35351,3PPCodedTest,2020-01-29T04:14:00.670+0000,2020-01-29T04:14:00.670+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,10,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35361,3PPCodedTest,2020-01-29T04:15:34.340+0000,2020-01-29T04:16:59.580+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,30,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35689,3PPCodedTest,2020-01-29T05:00:05.920+0000,2020-01-29T05:00:05.920+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,30,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35694,3PPCodedTest,2020-01-29T05:00:37.020+0000,2020-01-29T05:00:37.020+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,10,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35820,3PPCodedTest,2020-01-29T06:53:17.490+0000,2020-01-29T06:53:17.490+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,30,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35912,3PPCodedTest,2020-01-29T07:04:40.690+0000,2020-01-29T07:04:40.690+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,20,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35947,3PPCodedTest,2020-01-29T07:09:23.460+0000,2020-01-29T07:09:23.460+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,20,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,35982,3PPCodedTest,2020-01-29T07:13:34.430+0000,2020-01-29T07:13:34.430+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,30,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,36131,3PPCodedTest,2020-01-29T07:32:01.320+0000,2020-01-29T07:32:01.320+0000,Prasad Kumar Koukuntla,takeit@takehome.com,Apt#5,156th Ave,Bellevue,null,98007,4252887652,10,US,null,US-WA,null,0,null,null,false,null,2020,1,1
1,36355,tg54t5,2020-02-05T12:33:59.140+0000,2020-02-05T12:33:59.140+0000,Sergey Ahn,sahn@actsoft.com,Actsoft street 2,Actsoft street 1,Tampa,null,12345,2345678910,20,US,null,US-FL,null,0,null,null,false,null,2020,1,1
